<a href="https://colab.research.google.com/github/rivera-squared/Educacion_Machine_Learning/blob/main/educacion_DeepLearning_newmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Model


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from tensorflow.keras.optimizers import SGD

In [3]:
# Import the SGD optimizer
from keras.callbacks import EarlyStopping

In [4]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [93]:
# New data set with 'cambio_matricula' column
x = pd.read_csv('https://raw.githubusercontent.com/edghero/data/main/escuelas_nuevas.csv').dropna()

In [94]:
x['cambio_matricula'] = pd.to_numeric(x['cambio_matricula'], errors = 'coerce')

In [69]:
# Setting index
x = x.set_index('escuela')

In [95]:
# renaming nivel to match DE's current naming convention. I NEED TO GO OVER INTERLOCKING ONE BY ONE TO RECLASS THEM PROPERLY
x['nivel'] = np.where(x['nivel'] == 'elemental', 'primario',
                      np.where(x['nivel'] == 'intermedia','primario',
                               np.where(x['nivel'] == 'especializada','todos los niveles',
                                        np.where(x['nivel'] == 'superior', 'secundario',
                                                 np.where(x['nivel'] == 'interlocking','todos los niveles','otro')))))

In [97]:
primario = x[x['nivel'] == 'primario']
primario = primario[['consolidada','matricula','cambio_matricula']]
primario

,consolidada,matricula,cambio_matricula
1,0,473,-0.11
2,0,332,1.24
3,0,473,-0.11
6,0,294,0.15
7,0,428,-0.04
...,...,...,...
1085,0,309,-0.04
1087,0,229,-0.39
1088,0,222,-0.13
1089,0,387,-0.04


In [99]:
X = primario.drop(columns = 'consolidada')
y = primario['consolidada']

In [100]:
predictors_primario = X
target_primario = to_categorical(y)

First Deep Learning Model for 'escuelas primarias' with 'Adam' optimizer

In [78]:
early_stopping_monitor = EarlyStopping(patience=5)

In [102]:
# Set up the model: model
n_cols = predictors_primario.shape[1]
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=(n_cols,))) # Add the first layer
model.add(Dense(30, activation="relu")) # Add the second layer
model.add(Dense(50, activation="sigmoid")) # Add the second layer
model.add(Dense(2, activation = 'softmax')) # Add the output layer; needed to add softmax activation since this method is for classifiication problem set

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ['accuracy'])

model.fit(predictors_primario, target_primario, validation_split = 0.3, epochs = 50, callbacks = [early_stopping_monitor])

Epoch 1/50
15/15 [==============================] - 1s 17ms/step - loss: 0.6483 - accuracy: 0.6083 - val_loss: 0.5424 - val_accuracy: 0.7551
Epoch 2/50
15/15 [==============================] - 0s 5ms/step - loss: 0.5895 - accuracy: 0.6674 - val_loss: 0.5902 - val_accuracy: 0.7653
Epoch 3/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5752 - accuracy: 0.7068 - val_loss: 0.5202 - val_accuracy: 0.7449
Epoch 4/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5782 - accuracy: 0.6849 - val_loss: 0.4968 - val_accuracy: 0.7653
Epoch 5/50
15/15 [==============================] - 0s 4ms/step - loss: 0.5604 - accuracy: 0.7046 - val_loss: 0.5037 - val_accuracy: 0.7857
Epoch 6/50
15/15 [==============================] - 0s 5ms/step - loss: 0.5491 - accuracy: 0.7090 - val_loss: 0.5023 - val_accuracy: 0.7908
Epoch 7/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5381 - accuracy: 0.7352 - val_loss: 0.4720 - val_accuracy: 0.7857
Epoch 8/50
15/15 [=

In [103]:
# Calculate predictions
predictions_primario = model.predict(predictors_primario)

In [104]:
primario['predictions'] = predictions_primario[:,1]

In [105]:
primario['predictions'] = round(primario.predictions)

In [106]:
print(confusion_matrix(primario['consolidada'], primario['predictions']))
print(classification_report(primario['consolidada'], primario['predictions']))

[[391  52]
 [105 105]]
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       443
           1       0.67      0.50      0.57       210

    accuracy                           0.76       653
   macro avg       0.73      0.69      0.70       653
weighted avg       0.75      0.76      0.75       653



In [ ]:
from keras.models import load_model

In [ ]:
model.save('model_DL_adam.h5')

Deep Learning Model with SGD optimizer with different learning rates

In [107]:
early_stopping_monitor = EarlyStopping(patience=2)

In [ ]:
n_cols = predictors_primario.shape[1]

lr_to_test = [.000001, .001, .00025]

# loop over learning rates
for lr in lr_to_test:
  print('\n\nTesting model with learning rate: %f\n'%lr )
  model = Sequential()
  model.add(Dense(100, activation="relu", input_shape=(n_cols,))) # Add the first layer
  model.add(Dense(30, activation="relu")) # Add the second layer
  model.add(Dense(50, activation="relu")) # Add the second layer
  model.add(Dense(2, activation = 'softmax')) # Add the output layer; needed to add softmax activation since this method is for classifiication problem set
  my_optimizer = SGD(learning_rate=lr)
  model.compile(optimizer= my_optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
  #print("Loss function: " + model.loss)
  model.fit(predictors_primario, target_primario, validation_split=0.3, epochs= 50, callbacks=[early_stopping_monitor])


####Model with SGD as optimizer and learning rate of 0.00025

In [85]:
early_stopping_monitor = EarlyStopping(patience=5)

In [109]:
n_cols = predictors_primario.shape[1]

lr_to_test = [.00025]

# loop over learning rates
for lr in lr_to_test:
  print('\n\nTesting model with learning rate: %f\n'%lr )
  model = Sequential()
  model.add(Dense(100, activation="relu", input_shape=(n_cols,))) # Add the first layer
  model.add(Dense(30, activation="relu")) # Add the second layer
  model.add(Dense(50, activation="relu")) # Add the second layer
  model.add(Dense(2, activation = 'softmax')) # Add the output layer; needed to add softmax activation since this method is for classifiication problem set
  my_optimizer = SGD(learning_rate=lr)
  model.compile(optimizer= my_optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
  #print("Loss function: " + model.loss)
  model.fit(predictors_primario, target_primario, validation_split=0.3, epochs= 50, callbacks=[early_stopping_monitor])




Testing model with learning rate: 0.000250

Epoch 1/50
15/15 [==============================] - 1s 17ms/step - loss: 1.4886 - accuracy: 0.5295 - val_loss: 0.7784 - val_accuracy: 0.7245
Epoch 2/50
15/15 [==============================] - 0s 4ms/step - loss: 0.8499 - accuracy: 0.5974 - val_loss: 0.6219 - val_accuracy: 0.7245
Epoch 3/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6954 - accuracy: 0.6105 - val_loss: 0.5043 - val_accuracy: 0.7245
Epoch 4/50
15/15 [==============================] - 0s 4ms/step - loss: 0.8168 - accuracy: 0.6324 - val_loss: 0.6010 - val_accuracy: 0.7245
Epoch 5/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6261 - accuracy: 0.6586 - val_loss: 0.6668 - val_accuracy: 0.7245


In [110]:
# Calculate predictions
predictions_primario = model.predict(predictors_primario)

In [111]:
primario['predictions'] = predictions_primario[:,1]

In [112]:
primario['predictions'] = round(primario.predictions)

In [ ]:
primario

In [113]:
print(confusion_matrix(primario['consolidada'], primario['predictions']))
print(classification_report(primario['consolidada'], primario['predictions']))

[[443   0]
 [210   0]]
              precision    recall  f1-score   support

           0       0.68      1.00      0.81       443
           1       0.00      0.00      0.00       210

    accuracy                           0.68       653
   macro avg       0.34      0.50      0.40       653
weighted avg       0.46      0.68      0.55       653



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
